In [1]:
import numpy as np

# 设置numpy结果显示格式
np.set_printoptions(suppress=True, 
	# precision=20,    # 精确位数
	threshold=10,      # 每行最多打印多少数据
	linewidth=200)     # 宽度多少字符换行


def pca(X, k):
    # 样本中心化,axis=0是计算每一列的均值
    X = X - np.mean(X, axis=0)
    # 将X变为nxm（5x68040）形式
    X = X.T
    # 计算协方差矩阵5x5
    C_before = (1 / (X.shape[1] - 1)) * np.dot(X, X.T)
    # 计算特征值和对应的特征向量矩阵
    eigenvalues, eigenvectors = np.linalg.eig(C_before)
    # 求出最大的k个特征值的索引，np.argsort返回特征值从小到大排序后的数在原数组中的索引
    indexes = np.argsort(eigenvalues)[::-1][:k]  # [::-1]将数组倒序，[:k]取前k个值
    # 选择特征向量矩阵中列索引在indexes中的所有列构成最终的投影矩阵
    P = eigenvectors[:, indexes]
    # 将特征向量转为沿行方向排列，P：5x32
    P = P.T
    # 计算降维结果Y=PX（P：5x32，X：32x68040）
    Y = np.dot(P, X)
    # 求降维之后的矩阵协方差
    # C_after = np.cov(Y, rowvar=1)
    # 先把Y变成68040x5，否则不好直接减去均值
    Y = Y.T
    Y = Y - np.mean(Y, axis=0)
    # 再把Y转回5x68040
    Y = Y.T
    C_after = (1 / (Y.shape[1] - 1)) * np.dot(Y, Y.T)
    
    return C_before, Y, C_after

In [2]:
# 导入数据集
ColorHist = np.loadtxt('ColorHistogram.asc', usecols=range(1, 33))
print(ColorHist.shape, '\n', ColorHist)

(68040, 32) 
 [[0.002188 0.       0.       ... 0.       0.       0.      ]
 [0.002917 0.315417 0.188854 ... 0.326562 0.133958 0.011771]
 [0.000313 0.009825 0.008978 ... 0.001701 0.004701 0.288647]
 ...
 [0.163362 0.38669  0.166258 ... 0.001771 0.000834 0.000521]
 [0.031892 0.119415 0.045017 ... 0.001563 0.000104 0.000833]
 [0.004899 0.0022   0.001463 ... 0.006875 0.000208 0.000417]]


In [3]:
# 降至5维
C_before, Y, C_after = pca(ColorHist, 5)
# 输出相关结果.
print('原始数据的协方差矩阵', C_before.shape, '\n', C_before, '\n\nPCA降维后的数据', Y.shape, '\n', Y, '\n\n降维后数据的协方差矩阵', C_after.shape, '\n', C_after)

原始数据的协方差矩阵 (32, 32) 
 [[ 0.01069675  0.00312396 -0.00065764 ...  0.00009893 -0.00006514 -0.00018398]
 [ 0.00312396  0.01495444  0.004956   ...  0.00015231 -0.00001139 -0.00023112]
 [-0.00065764  0.004956    0.01198136 ...  0.00008408  0.00019491  0.00013184]
 ...
 [ 0.00009893  0.00015231  0.00008408 ...  0.00084625  0.00038103  0.00015934]
 [-0.00006514 -0.00001139  0.00019491 ...  0.00038103  0.00078702  0.00043224]
 [-0.00018398 -0.00023112  0.00013184 ...  0.00015934  0.00043224  0.00185667]] 

PCA降维后的数据 (5, 68040) 
 [[-0.31544492 -0.21863173 -0.34753955 ... -0.27255939 -0.04559854  0.01109668]
 [-0.0179496  -0.03341884 -0.00672474 ... -0.09667564 -0.00850077  0.01036008]
 [ 0.1383116  -0.11073215  0.06563212 ... -0.15275086  0.06349718  0.26553707]
 [ 0.38837362 -0.12878784  0.42596456 ... -0.15610792 -0.05805846 -0.07390119]
 [ 0.03970076 -0.01361756  0.04070145 ... -0.00383758  0.04464355  0.04824296]] 

降维后数据的协方差矩阵 (5, 5) 
 [[ 0.03101495 -0.          0.         -0.          0. 